# Node Embeddings with Properties

### Imports
Importieren der benötigten Bibliotheken


In [36]:
from neo4j import GraphDatabase
import graphdatascience

### Connection
Dabei wird eine Verbindung zur Datenbank aufgebaut und die Graph Data Science Library initialisiert.

In [60]:
gds = graphdatascience.GraphDataScience("neo4j://localhost:7687", auth=("neo4j", "bachelorarbeit"))
gdsdriver = GraphDatabase.driver("neo4j://localhost:7687", auth=("neo4j", "bachelorarbeit")).session(database="neo4j")

In [62]:
def run_query(query, parameters=None):
    with gdsdriver as session:
        result = session.run(query, parameters)
        return [record.data() for record in result]
    
check_if_exist_query = """
RETURN gds.graph.exists('propertyGraph') 
"""

# Projection query
projection_query = """
MATCH (source)-[r:HAS_START|HAS_END|PARTICIPATED_IN|USED_IN]->(target)
WHERE source:Trip
WITH gds.graph.project(
  'propertyGraphWithProperties',
  source,
  target,
  {
  sourceNodeProperties: source { year: coalesce(source.validFrom.year, 0), month: coalesce(source.validFrom.month, 0), day: coalesce(source.validFrom.day, 0),
  hour: coalesce(source.validFrom.hour, 0), minute: coalesce(source.validFrom.minute, 0), weekday: coalesce(source.validFrom.dayOfWeek, 0)},
  targetNodeProperties: target { year: coalesce(target.validFrom.year, 0), month: coalesce(target.validFrom.month, 0), day: coalesce(target.validFrom.day, 0),
  hour: coalesce(target.validFrom.hour, 0), minute: coalesce(target.validFrom.minute, 0), weekday: coalesce(target.validFrom.dayOfWeek, 0)}
  },
  {undirectedRelationshipTypes: ['*']}
) as g
RETURN g.graphName AS graph, g.nodeCount AS nodes, g.relationshipCount AS rels
"""

run_query(projection_query)

[{'graph': 'propertyGraphWithProperties', 'nodes': 727085, 'rels': 2906174}]

In [63]:
G = gds.graph.get("propertyGraphWithProperties")

result = gds.fastRP.write.estimate(
    G,
    writeProperty="temporalEmbeddingWithProperties1",
    randomSeed = 42,
    embeddingDimension=128,
    propertyRatio = 0.2,
    featureProperties= ['day', 'hour', 'minute', 'weekday'],
    iterationWeights=[1.0, 0.8, 0.8, 0.8])

print(result)

requiredMemory                                                 599 MiB
treeView             Memory Estimation: 599 MiB\n|-- algorithm: 599...
mapView              {'memoryUsage': '599 MiB', 'name': 'Memory Est...
bytesMin                                                     628207232
bytesMax                                                     628207232
nodeCount                                                       727085
relationshipCount                                              2906174
heapPercentageMin                                                  0.2
heapPercentageMax                                                  0.2
Name: 0, dtype: object


In [65]:
result = gds.fastRP.write(
    G,
    writeProperty="temporalEmbeddingWithProperties1",
    randomSeed = 42,
    embeddingDimension=128,
    propertyRatio = 0.2,
    featureProperties= ['day', 'hour', 'minute', 'weekday'],
    iterationWeights= [1.0, 0.8, 0.8, 0.8]
)

print(result)

FastRP:   0%|          | 0/100 [00:00<?, ?%/s]

nodeCount                                                           727085
nodePropertiesWritten                                               727085
preProcessingMillis                                                      1
computeMillis                                                         4103
writeMillis                                                         203501
configuration            {'writeProperty': 'temporalEmbeddingWithProper...
Name: 0, dtype: object


In [66]:
G.drop()
gdsdriver.close()